# SHare Model

In [1]:
import tensorflow as tf
print(tf.__version__)

2022-06-10 18:22:37.665793: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-10 18:22:37.665868: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.8.0


## Library Used

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.regularizers import l2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
df = pd.read_csv('Dataset.csv')
df.head()

,text,pasal
0,"Pak A sering pegang pegang tangan, kadang pega...",1
1,lama lama dia tarik tanganku baru baring di pa...,1
2,pake jilbab syar'i dan gapernah sentuhan sama ...,0
3,"saya ditarik"" hampir masuk di ruangannya dosen...",1
4,saat mengajar dia mendekati teman saya dengan ...,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    85 non-null     object
 1   pasal   85 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ KB


In [5]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import nltk
# nltk.download('punkt') # download tokenizer for first time using nltk only

factory = StemmerFactory()
stemmer_factory = factory.create_stemmer()
factory = StopWordRemoverFactory()
stopword_factory = factory.create_stop_word_remover()
def preprocessing(word):
    steaming = stemmer_factory.stem(word)
    stopword = stopword_factory.remove(steaming)
    final_token = nltk.tokenize.word_tokenize(stopword)
    return final_token

df.text = df.text.map(preprocessing)
df.head()

,text,pasal
0,"[pegang, pegang, tangan, kadang, pegang, paha]",1
1,"[tarik, tangan, baring, pangku]",1
2,"[pake, jilbab, syar, gapernah, sentuh, lawan, ...",0
3,"[tarik, ruang, dosen]",1
4,"[ajar, teman, sengaja, wajah]",1


In [6]:
max_len = 0


for text in df["text"] :
  if len(text) > max_len :
    max_len = len(text)


for text in df["text"] :
  text += [' '] * (max_len - len(text))

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.text, df.pasal, test_size=0.2, random_state=42)

In [8]:
len(x_train), len(x_test), len(y_train), len(y_test)

(68, 17, 68, 17)

In [9]:
def embedding(sentence_list):
    for sentence in sentence_list:
        for word in sentence:
            data_tensor = tf.convert_to_tensor(word, dtype=tf.string)
            return data_tensor

In [10]:
x_train = embedding(x_train)
x_test = embedding(x_test)

2022-06-10 18:23:18.291799: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-10 18:23:18.291853: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-10 18:23:18.291888: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (FaN0016): /proc/driver/nvidia/version does not exist


In [11]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [12]:
import tensorflow_hub as hub
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=False)

In [17]:
model = tf.keras.models.Sequential([
                                        hub_layer, 
                                        layers.Dense(1024, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)), 
                                        layers.Dropout(0.3), 
                                        layers.Dense(1, activation='softmax')
])

In [14]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
history = model.fit(x_train, y_train,
          epochs = 500,
          validation_data = (x_test, y_test))

IndexError: list index out of range